# Illumina Genome Assembly

We are now going to use the assembler velvet [https://www.ebi.ac.uk/~zerbino/velvet/](https://www.ebi.ac.uk/~zerbino/velvet/) to assemble the Illumina reads. Our Illumina reads are from the same sample we used to generate the PacBio data.

## Generating an Illumina assembly with velvet

Creating a genome assembly using velvet is a two stage process:

* First, the command `velveth` is used to generate the k-mers from the input data
* Second, the command `velvetg` is used to build the de Bruijn graph and find the optimal path through the graph

To assemble the data, start with the command:

In [ ]:
velveth k.assembly.49 49 -shortPaired -fastq IT.Chr5_1.fastq IT.Chr5_2.fastq

The input option `k.assembly.49` is the name of the directory where the results are to be written. The vallue `49` is the k-mer size. The other options specify the type of the input data (`-shortPaired`) and `-fastq` is used to specify the fastq files that contain the sequencing reads. 

To see all possible options for velveth use:

In [ ]:
velveth

Now use velvetg to build the graph and find the path through the graph (similar to what we did manually in the previous section):

In [ ]:
velvetg k.assembly.49 -exp_cov auto -ins_length 350

The first parameter `k.assembly.49` specifies the working directory as created with the `velveth` command. The second `-exp_cov auto` instructs velvet to find the median read coverage automatically rather than specifying it yourself. Finally, `-ins_length` specifies the insert size of the sequencing library used. There is a lot of output printed to the screen, but the most important information is the last line:

`Final graph has 1455 nodes and n50 of 7527, max 38045, total 1364551, using 700301/770774 reads.
(Your exact result might differ depending on the velvet version used - don’t worry).`

This gives you a quick idea of the result. 

* 1455 nodes are in the final graph. 
* An n50 of 7527 means that 50% of the assembly is in contigs of at least 7527 bases, it is the median contig size. This number is most commonly used as an indicator of assembly quality. The higher, the better! (but not always!) 
* Max is the length of the longest contig. 
* Total is the size of the assembly, here 1346kb. 
* The last two numbers tell us how many reads were used from the 7.7 million pairs input data.

To see all possible options for velvetg use:

In [ ]:
velvetg

Now let's try to improve the quality of the assembly by varying some of the input parameters to velvet.  Two parameters that can play a role in improving the assembly are `-cov_cutoff` and `-min_contig_lgth`. 
 
Using the `-cov_cutoff` parameter means that nodes with less than a specific k-mer count are removed from the graph.

Using the `-min_contig_lgth` parameter means that contigs with less than a specific size are removed from the assembly.

Try re-running the assembly with a kmer of 49 and using a `-cov_cutoff` of 5 and `-min_contig_lgth` of 200.

In [ ]:
velvetg k.assembly.49 -exp_cov auto -ins_length 350 -min_contig_lgth 200 -cov_cutoff 5

Note as we are not changing the k-mer size, we do not need run the `velveth` command again.

Generally, the k-mer size has the biggest impact on assembly results. Let us make a few other assemblies for different k-mer sizes i.e. 55, 41. Here is the example for k-mer length of 55.

In [ ]:
velveth k.assembly.55 55 -shortPaired -fastq IT.Chr5_1.fastq IT.Chr5_2.fastq

In [ ]:
velvetg k.assembly.55 -exp_cov auto -ins_length 350 -min_contig_lgth 200 -cov_cutoff 5

**Note:** If you find that you are having trouble running the velvet assemblies or if it is running for longer than 10-15 mins then quit the command (Ctrc-C). A pre-generated set of Illumina assemblies can be found at:

In [ ]:
ls ~/course_data/assembly/data/assembly_backup

## Assembly metrics

All the assembly results are written into the directory you specified with the `velvet` commands, e.g. `k.assembly.41`,`k.assembly.49`,`k.assembly.55`. The final contigs are written to a file called `contigs.fa`. The `stats.txt` file holds some information about each contig, its length, the coverage, etc. The other files contain information for the assembler.

Another way to get more assembly statistics is to use a program called `assembly-stats`. It displays the number of contigs, the mean size and a lot of other useful statistics about the assembly. These numbers can be used to assess the quality of your assemblies and help you pick the "best" one.

Type:

In [ ]:
assembly-stats k.assembly*/*.fa

![Output from assembly stats](images/assembly_stats_output.png "Output from assembly stats")

Write down the results for each assembly made using different k-mer sizes.

![Velvet assembly stats](images/velvet_stats.png "Velvet assembly stats")

**Question:** What is the best choice for k?

We want to choose the set of parameters that produce the assembly where the n50, average contig size and the largest contigs have the highest values, while contig number is the lowest.

You will notice another statistic produced by assembly-stats is N_count, what does the N_count mean?

As we know, DNA templates can be sequenced from both ends, resulting in mate pairs. Their outer distance is the insert size. Imagine mapping the reads back onto the assembled contigs. In some cases the two mates don’t map onto the same contig. We can use those mates to scaffold the two contigs e.g. orientate them to each other and put N’s between them, so that the insert size is correct, if enough mate pairs suggest that join. Velvet does this automatically (although you can turn it off). The number of mates you need to join two contigs is defined by the parameter `-min_pair_count`.

Here is the description:

`-min_pair_count <integer>: minimum number of paired end connections to justify the scaffolding of two long contigs (default: 5)`

Here is a schema:

![Assembly scaffolds](images/schema.png "Assembly scaffolds")

It might be worth mentioning, that incorrect scaffolding is the most common source of error in assembly (so called mis-assemblies). If you lower the min_pair_count too much, the likelihood of generating errors increases.

Other errors are due to repeats. In a normal assembly one would expect that the repeats are all collapsed, if they are smaller than the read length. If the repeat unit is smaller than the insert size, than it is possible to scaffold over it, leaving the space for the repeats with N’s.

To get the statistic for the contigs, rather than scaffolds (supercontigs), you can use `seqtk` to break the scaffold at any stretch of N’s with the following commands:

In [ ]:
seqtk cutN -n1 k.assembly.41/contigs.fa > assembly.41.contigs.fasta

In [ ]:
assembly-stats assembly.41.contigs.fasta

In [ ]:
seqtk cutN -n1 k.assembly.49/contigs.fa > assembly.49.contigs.fasta

In [ ]:
assembly-stats assembly.49.contigs.fasta

In [ ]:
seqtk cutN -n1 k.assembly.55/contigs.fa > assembly.55.contigs.fasta

In [ ]:
assembly-stats assembly.55.contigs.fasta

**Question:** How does the contig N50 compare to the scaffold N50 for each of your assemblies?

![Output from assembly stats for velvet contigs](images/n50_comparisons.png "Output from assembly stats for velvet contigs")

Congratulations you have sucessfully created a genome assembly using Illumina sequence data. Now move on to the next section: [Assembly estimation](assembly_estimation.ipynb)